In [4]:
import numpy as np
from math import exp, cos, sqrt, log
from scipy import integrate

# --------------------------------------------------
# Definición de funciones
# --------------------------------------------------

def f(x):
    return np.exp(-x) * np.cos(5*x)

def g(x):
    return np.sqrt(x * np.log(x + 1))

# Intervalos
a_f, b_f = 0.0, 2.0
a_g, b_g = 0.0, 1.0

# --------------------------------------------------
# Métodos de integración numérica
# --------------------------------------------------

def punto_medio(f, a, b, n):
    h = (b - a) / n
    suma = 0.0
    for i in range(n):
        xm = a + (i + 0.5) * h
        suma += f(xm)
    return h * suma

def trapecio(f, a, b, n):
    h = (b - a) / n
    suma = 0.5 * (f(a) + f(b))
    for i in range(1, n):
        suma += f(a + i * h)
    return h * suma

def simpson(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n debe ser par para Simpson")
    h = (b - a) / n
    suma = f(a) + f(b)
    for i in range(1, n):
        coef = 4 if i % 2 != 0 else 2
        suma += coef * f(a + i * h)
    return h * suma / 3

# --------------------------------------------------
# Valor de referencia (alta precisión)
# --------------------------------------------------

I_f_ref, _ = integrate.quad(f, a_f, b_f, epsabs=1e-12)
I_g_ref, _ = integrate.quad(g, a_g, b_g, epsabs=1e-12)

print("VALOR DE REFERENCIA")
print("If =", I_f_ref)
print("Ig =", I_g_ref)
print()

# --------------------------------------------------
# Aproximaciones para n = 4 y n = 8
# --------------------------------------------------

for n in [4, 8]:
    print(f"APROXIMACIONES PARA n = {n}")
    pm = punto_medio(f, a_f, b_f, n)
    tr = trapecio(f, a_f, b_f, n)
    si = simpson(f, a_f, b_f, n)

    print(" Punto medio :", pm, " Error:", abs(pm - I_f_ref))
    print(" Trapecio    :", tr, " Error:", abs(tr - I_f_ref))
    print(" Simpson     :", si, " Error:", abs(si - I_f_ref))
    print()

# --------------------------------------------------
# Orden empírico de convergencia
# --------------------------------------------------

print("ORDEN EMPÍRICO DE CONVERGENCIA")

E4 = abs(trapecio(f, a_f, b_f, 4) - I_f_ref)
E8 = abs(trapecio(f, a_f, b_f, 8) - I_f_ref)
orden_trap = np.log(E4 / E8) / np.log(2)

E4 = abs(simpson(f, a_f, b_f, 4) - I_f_ref)
E8 = abs(simpson(f, a_f, b_f, 8) - I_f_ref)
orden_simp = np.log(E4 / E8) / np.log(2)

print(" Orden trapecio:", orden_trap)
print(" Orden Simpson :", orden_simp)
print()

# --------------------------------------------------
# Simpson y trapecio con n = 16
# --------------------------------------------------

print("n = 16")
print(" Simpson :", simpson(f, a_f, b_f, 16))
print(" Trapecio:", trapecio(f, a_f, b_f, 16))
print()

# --------------------------------------------------
# Romberg
# --------------------------------------------------

def romberg(f, a, b):
    R = np.zeros((3,3))
    R[0,0] = trapecio(f, a, b, 1)
    R[1,0] = trapecio(f, a, b, 2)
    R[2,0] = trapecio(f, a, b, 4)

    R[1,1] = R[1,0] + (R[1,0] - R[0,0]) / 3
    R[2,1] = R[2,0] + (R[2,0] - R[1,0]) / 3
    R[2,2] = R[2,1] + (R[2,1] - R[1,1]) / 15

    return R

R = romberg(f, a_f, b_f)
print("TABLA DE ROMBERG")
print(R)
print("R2,2 =", R[2,2])
print()

# --------------------------------------------------
# Simpson adaptativo
# --------------------------------------------------

If_adapt, err, info = integrate.quad(f, a_f, b_f, epsabs=1e-6, full_output=1)

print("SIMPSON ADAPTATIVO")
print(" Valor:", If_adapt)
print(" Evaluaciones de f:", info['neval'])
print()

# --------------------------------------------------
# Integral Ig con diferentes métodos
# --------------------------------------------------

print("INTEGRAL Ig")

print(" Trapecio:", trapecio(g, a_g, b_g, 16))
print(" Simpson :", simpson(g, a_g, b_g, 16))

Ig_adapt, err, info = integrate.quad(g, a_g, b_g, epsabs=1e-6, full_output=1)
print(" Adaptativo:", Ig_adapt)
print(" Evaluaciones:", info['neval'])


VALOR DE REFERENCIA
If = 0.028670374130722733
Ig = 0.4391170392548742

APROXIMACIONES PARA n = 4
 Punto medio : 0.004312463340752762  Error: 0.02435791078996997
 Trapecio    : 0.06950106153369859  Error: 0.04083068740297585
 Simpson     : -0.08985708287107254  Error: 0.11852745700179527

APROXIMACIONES PARA n = 8
 Punto medio : 0.02435476203313821  Error: 0.004315612097584524
 Trapecio    : 0.03690676243722568  Error: 0.008236388306502949
 Simpson     : 0.026041996071734703  Error: 0.0026283780589880304

ORDEN EMPÍRICO DE CONVERGENCIA
 Orden trapecio: 2.309570103296987
 Orden Simpson : 5.494904685289581

n = 16
 Simpson : 0.028538762167834034
 Trapecio: 0.030630762235181994

TABLA DE ROMBERG
[[ 0.88644402  0.          0.        ]
 [ 0.54757549  0.43461932  0.        ]
 [ 0.06950106 -0.08985708 -0.12482218]]
R2,2 = -0.12482217644104156

SIMPSON ADAPTATIVO
 Valor: 0.028670374130722733
 Evaluaciones de f: 21

INTEGRAL Ig
 Trapecio: 0.4390247874162078
 Simpson : 0.43911698162970975
 Adapta